In [ ]:
############################### Organization ################################
#############################################################################

## Import some important libraries
import os, sys, json, time, cv2, skimage, numpy as np, tensorflow as tf, matplotlib.pyplot as plt
from PIL import Image, ImageDraw
from pathlib import Path
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score, accuracy_score

## Define root directory of the project
ROOT_DIR = "."
print("Check root directory: ", os.path.exists(ROOT_DIR), "\n") 
assert os.path.exists(ROOT_DIR), "ROOT_DIR does not exist"

## Import some important libraries of Mask R-CNN model 
sys.path.append(ROOT_DIR)
from config import Config
import utils as utils
import visualize
import model as modellib
from model import log
import dataset_preparation

## CPU and GPU verification (numbers) 
from tensorflow.python.client import device_lib
# print(device_lib.list_local_devices())

## Make a directory to save logs, structure of trained model, and trained weights files in h5 format
MODEL_DIR = os.path.join(ROOT_DIR, "Logs")
COCO_MODEL_PATH = os.path.join(ROOT_DIR, "mask_RCNN.h5")

## Download COCO trained weights from Releases if needed
if not os.path.exists(COCO_MODEL_PATH):
    utils.download_trained_weights(COCO_MODEL_PATH)

In [ ]:
############################# Model Configuration ###########################
#############################################################################

class CocoSynthConfig(Config):
    """Configuration for training on the box_synthetic dataset. Derives from the base Config class and overrides specific values.
    """
    ## Give the configuration a recognizable name
    NAME = "Hadi_33_"

    ## Train on 1 GPU and 1 image per GPU. Batch size is 1 (GPUs * images/GPU).
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1

    ## Number of classes (including background)
    NUM_CLASSES = 4    # 1 background + 3 cell types

    ## The size of training images originally was 512x512 in Mask R-CNN, However, we changed it to 64*64
    IMAGE_MIN_DIM = 960
    IMAGE_MAX_DIM = 960
    
    ## The confidence level (IoU) is 70% 
    DETECTION_MIN_CONFIDENCE = 0.70

    ## You can experiment with this number to see if it improves training. It is better to be 1000, but slower. 
    STEPS_PER_EPOCH = 1      

    ## This is how often validation is run. 
    ## If you are using too much hard drive space on saved models (in the MODEL_DIR), try making this value larger.
    VALIDATION_STEPS = 1
    
    ## Backbone of the CNN model: "resnet50", "resnet101"  
    BACKBONE = "resnet101"
    IMAGE_RESIZE_MODE = "hadi_crop"

    RPN_ANCHOR_SCALES = (8, 16, 32, 64, 128)
    TRAIN_ROIS_PER_IMAGE = 32
    MAX_GT_INSTANCES = 50 
    POST_NMS_ROIS_INFERENCE = 500 
    POST_NMS_ROIS_TRAINING = 1000 
    LEARNING_RATE = 0.001
    LEARNING_MOMENTUM = 0.9
    
    #############################################################################
    # If enabled, resizes instance masks to a smaller size to reduce memory load. Recommended when using high-resolution images.
    USE_MINI_MASK = True
    MINI_MASK_SHAPE = (56, 56)  # (height, width) of the mini-mask
    TRAIN_BN = False
    #############################################################################
    
config = CocoSynthConfig()
# config.display()

In [ ]:
################################# Datasets ##################################
#############################################################################

## Load train dataset (images and masks)
## I used the VIA 2.0.8 (VGG Image Annotator) software to annotate images, based on COCO standard format.
dataset_train = dataset_preparation.CocoLikeDataset()
dataset_train.load_data("Datasets/Phase1/train/Phase1_train.json", "Datasets/Phase1/train/images")
dataset_train.prepare()

## Load validation dataset (images and masks)
dataset_val = dataset_preparation.CocoLikeDataset()
dataset_val.load_data("Datasets/Phase1/val/Phase1_val.json", "Datasets/Phase1/val/images")
dataset_val.prepare()

## Print name of categories in the train dataset. The BG category refers BACKGROUND.
for name in [("Training", dataset_train)]:
    print("Categories:\n_________________")
    for i, info in enumerate(dataset_train.class_info):
        print("{:3}. {:50}".format(i, info["name"]))

## Showing n samples from Train and Validation dataset.
number_of_samples = 0                    # number of samples in each group
for name, dataset in [("Training", dataset_train), ("Validation", dataset_val)]:
    print("=================>", "Number of", name, "images:\t", "{}".format(len(dataset.image_ids)))
    image_ids = dataset.image_ids[0 : number_of_samples]   
    for image_id in image_ids:
        image = dataset.load_image(image_id)
        mask, class_ids = dataset.load_mask(image_id)
        visualize.display_top_masks(image, mask, class_ids, dataset.class_names)
        bbox = utils.extract_bboxes(mask)
        print("image_id ", image_id, dataset.image_reference(image_id))
        log("image", image)
        log("mask", mask)
        log("class_ids", class_ids)
        log("bbox", bbox)
        visualize.display_instances(image, bbox, mask, class_ids, dataset.class_names, size_of_gt =len(bbox), figsize=(10,10))

In [ ]:
################## Model initializing ##################
########################################################

model = modellib.MaskRCNN(mode = "training", config = config, model_dir = MODEL_DIR)

## Which weights to start with?
init_with = "imagenet"           # imagenet, coco, or last

if init_with == "imagenet":
    model.load_weights(model.get_imagenet_weights(), by_name = True)
elif init_with == "coco":
    model.load_weights(COCO_MODEL_PATH, by_name = True, exclude=["mrcnn_class_logits", "mrcnn_bbox_fc", "mrcnn_bbox", "mrcnn_mask"])
elif init_with == "last":
    ## Load the last model you trained and continue training
    model_path = str(Path(ROOT_DIR)/"Logs"/"hadi_33_20190819T1103/mask_rcnn_hadi_33__1200.h5")
    assert model_path != "", "Provide path to trained weights"
    model.load_weights(model_path, by_name = True)

###################### Train ######################
###################################################

start_train = time.time()
model.train(dataset_train, dataset_val, 
            learning_rate=config.LEARNING_RATE,    # fine train:     learning_rate=config.LEARNING_RATE / 10,
            epochs=200,
            layers="all")
end_train = time.time()
hours = round((end_train - start_train) / 1, 2)
print("\n", f"Training took {hours} seconds")

In [ ]:
###################### Test ######################
##################################################

class InferenceConfig(CocoSynthConfig):
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1
    IMAGE_MIN_DIM = 960
    IMAGE_MAX_DIM = 960
    DETECTION_MIN_CONFIDENCE = 0.70

inference_config = InferenceConfig()
## Recreate the model in inference mode
model = modellib.MaskRCNN(mode = "inference", config = inference_config, model_dir = MODEL_DIR)
## Get path to saved weights (Either set a specific path or find last trained weights)
# model_path = str(Path(ROOT_DIR)/"Logs/hadi_33_20190819T1103/mask_rcnn_hadi_33__1200.h5")    # trained by 512*512 images
model_path = str(Path(ROOT_DIR)/"Logs/hadi_33_20190912T1518/mask_rcnn_hadi_33__1102.h5")    # trained by 960*960 images
assert model_path != "", "Provide path to trained weights"
print("Loading weights from ", model_path)
model.load_weights(model_path, by_name = True)

##################################################

real_test_dir = "Datasets/Phase1/test/images"
image_paths = []
image_id = 0
dataset_val = dataset_preparation.CocoLikeDataset()
dataset_val.load_data("Datasets/Phase1/test/Phase1_test.json", "Datasets/Phase1/test/images")
dataset_val.prepare()
print("=================================")
for filename in os.listdir(real_test_dir):
    print(filename)
    if os.path.splitext(filename)[1].lower() in [".png", ".jpg", ".jpeg", ".tif"]:
        image_paths.append(os.path.join(real_test_dir, filename))
for image_path in image_paths:
    img = skimage.io.imread(image_path)
    if len(img.shape) == 2:
        img = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)
    img_arr = np.array(img)
    results = model.detect([img_arr], verbose = 0)
    Result_pred = results[0]
#     for i in Result_pred:
#         print(i, Result_pred[i].shape)
    print("\n#################################### Detections ####################################\n")
    print("Predicted class_ids =\n", Result_pred["scores"])
    visualize.display_instances(img, Result_pred["rois"], Result_pred["masks"], Result_pred["class_ids"],
                                dataset_train.class_names, Result_pred["scores"], title = "Detections", figsize = (10,10),
                                edgecolor = "red", size_of_gt = len(Result_pred["rois"]))
    print("#################################### Ground Truth #########################################")
    image = dataset_val.load_image(image_id)
    mask, class_ids = dataset_val.load_mask(image_id)
    bbox = utils.extract_bboxes(mask)
    gt_box = bbox
    gt_mask = mask
    gt_class_id = class_ids
    visualize.display_instances(image, bbox, mask, class_ids, dataset.class_names, title = "Ground Truth", 
                                figsize = (10,10), edgecolor = "darkgreen", size_of_gt = len(gt_box))
    image_id = image_id + 1
    print("###################################### Mix ############################################")
    visualize.display_differences(img, gt_box, gt_class_id, gt_mask, Result_pred["rois"], Result_pred["class_ids"], 
                                  Result_pred["scores"], Result_pred["masks"], dataset_train.class_names, 
                                  title = "Ground Truth and Derection", ax = None, show_mask = True, show_box = True, 
                                  iou_threshold = 0.5, score_threshold = 0.5, size_of_gt =len(gt_box), edgecolor = "darkgreen")
####################################### mAP ##########################################
    Result_metrics = utils.compute_ap(gt_box, gt_class_id, gt_mask, Result_pred["rois"], Result_pred["class_ids"], 
                                      Result_pred["scores"], Result_pred["masks"], iou_threshold = 0.7)
#     print("********* P =", np.mean(Result_metrics[1][1:-2]))
#     print("********* R =", np.mean(Result_metrics[2][1:-2]))
#     print("********* F1-score =", (2 * P * R) / (P + R), "\n")
#     print("********* Precision =", Result_metrics[1].shape, "\n", Result_metrics[1])
#     print("********* Recall =", Result_metrics[2].shape, "\n", Result_metrics[2])
#     print("********* Overlaps =", Result_metrics[3].shape, "\n", Result_metrics[3])
    print("********* mAP =", Result_metrics[0])
    visualize.plot_precision_recall(Result_metrics[0], Result_metrics[1], Result_metrics[2])

In [ ]:
############################### Confusion Matrix #####################################
######################################################################################

print(__doc__)
################################# Manually counted ###################################
# GT_N11 = np.array([0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,2,2])
# Detection_N11 = np.array([1,2,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,3])
# GT_C11 = np.array([1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,
#                    2,2,2,2,2,2,2,2,2,2,2,2,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3])
# Detection_C11 = np.array([0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,
#                           0,1,1,1,1,2,3,3,3,3,3,3,0,0,2,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3])
GT_P11 = np.array([1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,2,2,2,2,2,2,2,3,3,3,3,3,3,3,3])
Detection_P11 = np.array([1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,2,1,1,3,3,3,3,2,3,3,3,3,3,3,3])
######################################################################################

y_test = GT_P11
y_pred = Detection_P11
print(y_test.shape, y_pred.shape)
class_names = ["Live", "Intermediate", "    Pyknotic"]

def plot_confusion_matrix(y_true, y_pred, classes, normalize=False, title=None, cmap=plt.cm.Greens):
    if not title:
        if normalize:
            title = "Confusion matrix (Normalized)"
        else:
            title = "Confusion matrix"
    cm = confusion_matrix(y_true, y_pred)
    if normalize:
        cm = cm.astype("float") / cm.sum(axis = 1)[:, np.newaxis]
    fig, ax = plt.subplots(figsize=(14,7))
    im = ax.imshow(cm, interpolation = "nearest", cmap = cmap)
    ax.figure.colorbar(im, ax = ax)
    
    plt.title(title, fontsize = 30, pad = 50, fontweight = "bold", color = "blue")
    
    ax.set_xlabel("Predicted counts", fontsize = 25, fontweight = "bold")
    ax.set_ylabel("True counts", fontsize = 25, fontweight = "bold")
    ax.xaxis.set_label_position("top")
    ax.xaxis.set_label_coords(0.5, 1.25)
    ax.yaxis.set_label_coords(-0.5, 0.5)

#     plt.setp(ax.get_xticklabels(), rotation = 30, ha = "right", va = "center", rotation_mode = "anchor", style="italic")
    ax.set(xticks = np.arange(cm.shape[1]), yticks = np.arange(cm.shape[0]))
    ax.xaxis.tick_top() 
    ax.set_xticklabels(classes, size = 15, fontweight = "bold", color = "blue")
    ax.set_yticklabels(classes, size = 15, fontweight = "bold", color = "blue")
    ax.xaxis.set_tick_params(color="darkblue", width = 3, length = 8)
    ax.yaxis.set_tick_params(color="darkblue", width = 3, length = 8)

    fmt = ".3f" if normalize else "d"
    thresh = cm.max() / 2.
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            ax.text(j, i, format(cm[i, j], fmt), dict(size = 18), ha = "center", va = "center", 
                    fontname = "Arial", fontweight = "bold", color = "white" if cm[i, j] > thresh else "black")
    fig.tight_layout()
    fig.tight_layout(pad = 200, h_pad = 100, w_pad = 10, rect = (0,0,0.5,0.5))
    return ax

np.set_printoptions(precision = 3)
plot_confusion_matrix(y_test, y_pred, classes = class_names, title = "Confusion matrix")
plot_confusion_matrix(y_test, y_pred, classes = class_names, normalize = True, title = "Confusion matrix (Normalized)")
plt.show()

print("Precision =\t", precision_score(y_test, y_pred, average = "weighted"))
print("Recall =\t", recall_score(y_test, y_pred, average = "weighted"))
print("F1_score =\t", f1_score(y_test, y_pred, average = "weighted"))
print("Accuracy =\t", accuracy_score(y_test, y_pred, normalize = True))